In [ ]:
from pyspark.sql.functions import current_timestamp, col, lit, countDistinct, nullif
from capstone_pipeline.main import audit_log

table_name = "capstone.gold.daily_sales_summary"
log_path = "/Volumes/capstone/bronze/history"

In [ ]:
dfgold = (spark.table("capstone.gold.daily_sales_fact").groupBy("order_date", "region")
            .agg(
                sum("line_total").alias("total_revenue"),
                sum("quantity").alias("total_units_sold"),
                round((sum("line_total") / nullif(countDistinct("order_id"), lit(0))), 2).alias("avg_order_value"),
                countDistinct("order_id").alias("orders_count")
            )            
            .orderBy(col("order_date").desc(), col("region")))



In [ ]:
(dfgold
    .write
    .mode("overwrite")
    .format("delta")
    .saveAsTable("capstone.gold.daily_sales_summary"))

In [ ]:
audit_log(spark, table_name, log_path)